In [11]:
import os
import platform
import sys

def get_project_root() -> str:
    system = platform.system()
    return r"D:\Users\tcshore\Documents\Projects\Tangrams\Data" if system == "Windows" else "/home/tshore/Projects/tangrams-restricted/Data"

infile_path = os.path.join(get_project_root(), "wordprobseqs-inflected-small.tsv")
print("Will read file \"{}\".".format(infile_path), file=sys.stderr)

Will read file "D:\Users\tcshore\Documents\Projects\Tangrams\Data\wordprobseqs-inflected-small.tsv".


In [12]:
import csv
import pandas as pd

RESULTS_FILE_CSV_DIALECT = csv.excel_tab
# NOTE: "category" dtype doesn't work with pandas-0.21.0 but does with pandas-0.21.1
__RESULTS_FILE_DTYPES = {"DYAD": "category", "WORD": "category", "IS_TARGET": bool, "IS_OOV": bool,
				 "IS_INSTRUCTOR": bool, "SHAPE": "category", "ONLY_INSTRUCTOR": bool, "WEIGHT_BY_FREQ": bool}

def read_results_file(inpath: str, encoding: str) -> pd.DataFrame:
	print("Reading \"{}\" using encoding \"{}\".".format(inpath, encoding), file=sys.stderr)
	result = pd.read_csv(inpath, dialect=RESULTS_FILE_CSV_DIALECT, sep=RESULTS_FILE_CSV_DIALECT.delimiter,
						 float_precision="round_trip",
						 encoding=encoding, memory_map=True, dtype=__RESULTS_FILE_DTYPES)
	return result

cv_results = read_results_file(infile_path, "windows-1252")
print("Read {} cross-validation results for {} dyad(s).".format(cv_results.shape[0], cv_results["DYAD"].nunique()),
      file=sys.stderr)

Reading "D:\Users\tcshore\Documents\Projects\Tangrams\Data\wordprobseqs-inflected-small.tsv" using encoding "windows-1252".
Read 180880 cross-validation results for 2 dyad(s).


In [13]:
utts = cv_results.groupby(
		("CROSS_VALIDATION_ITER", "DYAD", "UTT_START_TIME", "UTT_END_TIME"), as_index=False)
print("Found {} utt token groups.".format(len(utts)), file=sys.stderr)

Found 297 utt token groups.


In [14]:
entity_ids = frozenset(cv_results["ENTITY"].unique())
print("Found {} unique entity IDs.".format(len(entity_ids)), file=sys.stderr)

Found 20 unique entity IDs.
